requires_grad = True表示在这个张量上开始追踪其所有操作

grad_fn返回一个与这些运算有关的对象，即这个张量是如何得到的

In [1]:
import torch
x = torch.ones(2, 2, requires_grad = True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


x是直接创建的，称为叶子节点，而y不是

In [3]:
print(x.is_leaf)
print(y.is_leaf)

True
False


In [4]:
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


可以用requires_grad_(True)的方式来in-place地改变requires_grad属性

In [5]:
a = torch.rand(1, 2)
print(x.requires_grad)
a.requires_grad_(True)
b = x ** 2
print(b)

True
tensor([[1., 1.],
        [1., 1.]], grad_fn=<PowBackward0>)


反向传播

这个推导很简单

In [6]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意，grad是累加的，因此如果要进行多次反向传播，记得先清零

清零用.grad.data.zeros_()

In [7]:
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_() # 将之前的梯度清零
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


XXX.backward()如果实在想传入参数，则必须传入一个和XXX同形的参数YYY

此时，相当于是先计算out = torch.sum(XXX * YYY)，此时out是标量，再对其进行求导

In [8]:
x = torch.tensor([1., 2., 3., 4.], requires_grad = True)
y = (x * 2).view(2, 2)
v = torch.tensor([1, 0.1, 0.01, 0.001]).view(2, 2)
y.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


中断梯度追踪的一个例子

要用到with torch.no_grad():代码块

In [9]:
x = torch.tensor(2., requires_grad = True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(y1)
print(y2)
print(y3)

y3.backward()
print(x.grad) # 4，y2的梯度不参与运算，所以答案是2 * y1 = 4

try:
    y2.backward() # 无法反向求导
except RuntimeError:
    print('RunTimeError')

tensor(4., grad_fn=<PowBackward0>)
tensor(8.)
tensor(12., grad_fn=<AddBackward0>)
tensor(4.)
RunTimeError


如果想修改tensor，又不希望被记录到梯度中，可以用tensor.data

In [10]:
x = torch.tensor(1., requires_grad = True)
y = x * 2
print(x.data)
print(x.data.requires_grad) # False，说明虽然仍为tensor，不过不在计算图里面了
x.data *= 100
y.backward()
print(x.grad)

tensor(1.)
False
tensor(2.)
